# Data Processing: Shapefile Processing
*Team 42: Elise Penn, Manish Vuyyuru, Victor Sheng, Yajaira Gonzalez*

Processes shapefiles to get population overlap for use in imputing data.

Needs to be in the folder /shapefiles in our github repo, where it has access to the shapefile data (too big to upload here.)

Takes ~5 hours to run all the way through.

In [ ]:
import numpy as np
import pickle
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from shapely.prepared import prep
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

In [ ]:
# Projection is lat/lon (unprojected/cylindrical equidistant)
# The proj.4 string:
# +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs

# Functions

In [ ]:
# some useful arrays to translate between naming conventions

election_year_list = np.array([1992, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 
                                   2014, 2016, 2018])
congress_ID_list = np.array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

state_names = np.array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 
               'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
               'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 
               'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 
               'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 
               'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 
               'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
               'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 
               'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 
               'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'])

state_abbrs = np.array(['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
              'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
              'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'])

state_fips = np.array([1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 
                       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
                       41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56])

In [ ]:
def read_shapefiles(election_years,verbose=True):
    """
    Reads in shapefiles from UCLA database (pre-2016) US Census TigerLine files (2016 on).
    data sources: http://cdmaps.polisci.ucla.edu/
                  https://www.census.gov/geo/maps-data/data/cbf/cbf_cds.html
    Note that file names must be in the same folder as the code with the following format:
        -pre-2016: districtShapes-NNN/districtsNNN.shp
        -2016 on: must be named t1_YYYY_us_cdNNN/tl_YYYY_us_cdNNN.shp
    ...where NNN is congress ID and YYYY is election year
    
    input:
        election_years -- (list) Elections years of the shape files you want to read. Even years only. 
        verbose -- (bool) set to True if you want it to print every file as it reads it in
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) with 
                       shapefiles stored in a column named 'shape'.
    """
    # read in the standard dictionary 
    district_df = pickle.load(open('../Datasets/master_index.p','rb'))
    district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff

    for election_year in election_years:
        # convert election year to "Nth Congress" 
        congress_ID = congress_ID_list[election_year_list==election_year][0]

        # pre-2016 files come from http://cdmaps.polisci.ucla.edu/
        # they are a bit cleaner
        if election_year < 2016:
            # read in the shapefile (must be named 'districtsN.shp' in a folder titled 'districtShapesN')
            shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)
            reader = shpreader.Reader(shpfilename) 
            districts = reader.records() # get full records
            geometries = reader.geometries() # get just the shape
            
            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon
                
                # 1) get the state abbr of the district
                if any(state_names==attr['STATENAME'].upper()): # filter out districts that aren't in states
                    ST = state_abbrs[state_names==attr['STATENAME'].upper()][0]    
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(attr['STATENAME'].upper()))
                    continue
                # 2) get the id of the district
                id_int = int(attr['DISTRICT'])
                if id_int == 0: # change at-large district ID from 0 to 1 to play nice with our indexing convention.
                    id_int = 1 # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                ID = '{0:02d}'.format(id_int)
                # 3) reformat the polygon if it intersects with itself
                if not poly.is_valid: 
                    # if it does, use buffer to correct this
                    if verbose:
                        print('The following polygons intersected with themselves. Attempting to buffer shape...')
                        print(ind)
                    poly = poly.buffer(0)
                # 4) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
            
        # 2016 and later are US Census Bureau Tiger Line Files 
        # (and they're a bit messier)
        if election_year >= 2016:
            # read in the shapefile (must be named 'tl_YYYY_us_cdN.shp' in a folder titled 't1_YYYY_us_cdN')
            shpfilename = 'tl_{1}_us_cd{0}/tl_{1}_us_cd{0}.shp'.format(congress_ID,election_year)
            reader = shpreader.Reader(shpfilename) 
            districts = reader.records() # get full records
            geometries = reader.geometries() # get just the shape

            # put the shapefiles into the standard dictionary
            for record in reader.records(): # loop over districts
                attr = record.attributes # dictionary of information about the district
                poly = record.geometry # coordinates of the district as a shapely polygon

                # check if polygon intersects with itself
                if not poly.is_valid: 
                    # if it does, use buffer to correct this
                    if verbose: 
                        print('The following polygons intersected with themselves. Attempting to buffer shape...')
                        print(ind)
                    poly = poly.buffer(0)
                
                # 1) get the state abbr of the district
                if any(state_fips==int(attr['STATEFP'])): # filter out districts that aren't in states
                    ST = state_abbrs[state_fips==int(attr['STATEFP'])][0]
                else: # pretty much just Washington, DC
                    print('{} is not a state.'.format(int(attr['STATEFP'])))
                    continue
                # 2) get the id of the district
                if (attr['CD{}FP'.format(congress_ID)]=='ZZ'):
                    print('District ID is ZZ, which indicates this is not a congressional district.')
                    continue
                else:
                    id_int = int(attr['CD{}FP'.format(congress_ID)])
                if id_int == 0: # change at-large district ID from 0 to 1 to play nice with our indexing convention.
                    # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                    id_int = 1
                ID = '{0:02d}'.format(id_int)
                # 3) reformat the polygon if it intersects with itself
                if not poly.is_valid: 
                    # if it does, use buffer to correct this
                    if verbose:
                        print('The following polygons intersected with themselves. Attempting to buffer shape...')
                        print(ind)
                    poly = poly.buffer(0)
                # 4) put it all together into an index
                ind = '{}_{}_{}'.format(ST, ID, election_year)
                if verbose:
                    print('{} was read in.'.format(ind))
                # put the polygon in the dictionary
                district_df.at[ind,'shape'] = poly
    
    return district_df

In [ ]:
# compute overlap percent between this district and last year's districts
def district_overlap(this_year, district_df, threshold_for_change=0.1):
    """
    Finds the fractional overlap between this year's district and the previous year's districts. 
    i.e., if a Florida's 1st district has changed its borders from 2014 to 2016, it may be made up of:
        20% of its area may come from district 2 in 2014
        40% of its area may come from district 3 in 2014
        10% of its area may come from district 4 in 2014
    
    input:
        this_year -- (int) Elections years of the districts you want to check. Even years only. 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       
                       fractional overlap stored in a column called 'fractional_overlap'.
                       Each district has a dictionary where the keys are the indicies of the previous
                       districts which overlap with our district, and the values are the fractional overlap.
                       
                       change from last year stored in a column called 'border_change'.
                       Statuses:
                           0 - this district has not changed at all since the previous year
                           1 - this district has either changed area > threshold since 
                               the previous year, or it is new.
    """
    # add a column
    if 'overlap_frac' not in district_df.columns:
        district_df['overlap_frac'] = [np.nan]*district_df.shape[0] # make a blank column
        district_df['overlap_frac'] = district_df['overlap_frac'].astype(object) # reassign to object so it can hold shapely stuff

    if 'border_change' not in district_df.columns:
        district_df['border_change'] = [np.nan]*district_df.shape[0] # make a blank column    
    
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]

        for ind,district in districts.iterrows(): # loop over districts in your current year
            overlap_dict = {}
            shape = district['shape']
            area = shape.area # area of this district
            
            # check if shapes intersect with themselves
            if not shape.is_valid: 
                # if they do, use buffer to correct this
                print('The following polygons intersected with themselves. Attempting to buffer shape...')
                print(ind)
                shape = shape.buffer(0)
                    
            for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
                shape_prev = district_prev['shape']
                
                # check if shapes intersect with themselves
                if not shape_prev.is_valid:
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    print(ind_prev)
                    shape_prev = shape_prev.buffer(0)

                # calculate frac overlap
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = np.around(overlap_area/area,decimals=3) # fractional overlap between new and old district
                                                                       # round to the thousandth

                if frac_overlap > 10**-3: # use threshold of 0.1% to avoid trivial changes
                    overlap_dict[ind_prev] = frac_overlap

            # make sure areas add up to 1.
            dict_sum = np.sum(list(overlap_dict.values()))
            if not np.isclose(dict_sum,1.,rtol=1e-03):
                # rescale so they add up to 1.
                print(overlap_dict)
                print('rescaled')
                print(dict_sum)
                overlap_dict = {key : np.around(val/dict_sum, decimals=3) 
                                for key,val in overlap_dict.items() if val/dict_sum > 10**-3}
                    
            print(ind)
            print(overlap_dict)
            district_df.at[ind, 'overlap_frac'] = overlap_dict

            # calculate the status of the new district
            max_frac = np.max(list(overlap_dict.values()))
            ind_prev = '{}{}'.format(ind[:6],prev_year) # same district, different year
            if ind_prev not in districts_prev.index: # the district is new this year
                print('new')
                district_df.at[ind, 'border_change'] = 1
            elif ind_prev not in overlap_dict.keys(): # the district existed last year, but is in a totally new location
                print('totally moved')
                district_df.at[ind, 'border_change'] = 1
            elif 1.-overlap_dict[ind_prev] > threshold_for_change: # the district existed last year, but the borders have moved
                print(1.-overlap_dict[ind_prev])
                print('changed')
                district_df.at[ind, 'border_change'] = 1
            else: # the district is essentially the same as last year
                print('same')
                district_df.at[ind, 'border_change'] = 0
            
    return district_df

In [ ]:
def get_centroid(district_df):
    """
    Finds the centroid of a district in lon,lat. 
    
    input: 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       with centroid stored in a column called 'centroid'.
                       The centroid is calculated as a lon,lat on a Cartesian plane.
                       It ignores spherical geometry. 
                       The centroid is stored as a tuple in the form (lon,lat)
    """
    # add column
    district_df['centroid'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['centroid'] = district_df['centroid'].astype(object) # reassign to object so it can hold shapely stuff
    
    for ind, district in district_df.iterrows():
        shape = district['shape']
        if pd.isnull(district['shape']): # if there's no shape, fill it with a nan
            district_df.at[ind, 'centroid'] = np.nan
        else:
            centroid = shape.centroid.coords
            district_df.at[ind, 'centroid'] = centroid # in units of lat/lon
            
    return district_df

In [ ]:
# compute overlap percent between this district and last year's districts
def population_overlap(this_year, district_df, threshold_for_change=0.1):
    """
    Estimates percent of population coming from previous district boundaries using 
    the equation:
    
    population_overlap = (overlap_area / area) * (1 / area_prev)
    
    Where overlap_area is the area of overlap between the new and old districts, 
    area is the area of the new district, and area_prev is the area of the previous 
    district. 
    
    This equation assumes that the voting population in each district is constant, 
    but area changes. Although each district has roughly 711,000 people, variations in 
    voter turnout between districts means that this assumption is false. 
    
    density_weighted_overlap is calculated for each previous district which intersects with 
    the new district. density_weigted_overlap is then scaled so that the sum of all 
    previous districts which intersect with the new district is 1. 
    
    input:
        this_year -- (int) Elections years of the districts you want to check. Even years only. 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       
                       population_overlap is stored in a column called 'population_overlap'.
                       Each district has a dictionary where the keys are the indicies of the previous
                       districts which overlap with our district, and the values are the population overlap.
                       
    """
    
    # add a column
    if 'population_overlap' not in district_df.columns:
        district_df['population_overlap'] = [np.nan]*district_df.shape[0] # make a blank column
        district_df['population_overlap'] = district_df['population_overlap'].astype(object) # reassign to object so it can hold shapely stuff

    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]

        for ind,district in districts.iterrows(): # loop over districts in your current year
            overlap_dict = {}
            shape = district['shape']
            area = shape.area # area of this district
            
            # check if shapes intersect with themselves
            if not shape.is_valid: 
                # if they do, use buffer to correct this
                print('The following polygons intersected with themselves. Attempting to buffer shape...')
                print(ind)
                shape = shape.buffer(0)
                    
            for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
                shape_prev = district_prev['shape']
                
                # check if shapes intersect with themselves
                if not shape_prev.is_valid:
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    print(ind_prev)
                    shape_prev = shape_prev.buffer(0)

                # calculate frac overlap
                area_prev = shape_prev.area
                density = 1./area_prev # assume population roughly the same, but area changes
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/area # fractional overlap between new and old district
                                                                       # round to the thousandth
                
                if frac_overlap > 10**-3: # use threshold of 0.1% to avoid trivial changes
                    population_overlap = frac_overlap*density
                    overlap_dict[ind_prev] = population_overlap

            # make sure areas add up to 1.
            dict_sum = np.sum(list(overlap_dict.values()))
            # rescale so they add up to 1.
            overlap_dict = {key : np.around(val/dict_sum, decimals=3) 
                            for key,val in overlap_dict.items() if val/dict_sum > 10**-3}

            print(ind)
            print(overlap_dict)
            district_df.at[ind, 'population_overlap'] = overlap_dict
            
    return district_df

In [ ]:
def inverse_population_overlap(this_year, district_df):
    """
    Exactly the same as population_overlap, except it gives each district_prev a dictionary
    where the keys are current year's districts, and the values are the estimated 
    population overlap of district_prev and district. 
    
    This allows you to impute values from (year) into (year-2) in your dataset. 
    
    """
    
    # add a column
    if 'inverse_population_overlap' not in district_df.columns:
        district_df['inverse_population_overlap'] = [np.nan]*district_df.shape[0] # make a blank column
        district_df['inverse_population_overlap'] = district_df['inverse_population_overlap'].astype(object) # reassign to object so it can hold shapely stuff
        
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]

        for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
            overlap_dict = {}
            shape_prev = district_prev['shape']
            area_prev = shape_prev.area # area of this district
            
            # check if shapes intersect with themselves
            if not shape_prev.is_valid: 
                # if they do, use buffer to correct this
                print('The following polygons intersected with themselves. Attempting to buffer shape...')
                print(ind_prev)
                shape_prev = shape_prev.buffer(0)
                    
            for ind,district in districts.iterrows(): # loop over districts in your current year
                shape = district['shape']
                
                # check if shapes intersect with themselves
                if not shape.is_valid:
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    print(ind)
                    shape = shape.buffer(0)

                # calculate frac overlap
                area = shape.area # area of current district
                density = 1./area # assume population roughly the same, but area changes
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/area_prev # fractional overlap between new and old district
                                                                       # round to the thousandth
                
                if frac_overlap > 10**-3: # use threshold of 0.1% to avoid trivial changes
                    population_overlap = frac_overlap*density
                    overlap_dict[ind] = population_overlap

            # make sure areas add up to 1.
            dict_sum = np.sum(list(overlap_dict.values()))
            # rescale so they add up to 1.
            overlap_dict = {key : np.around(val/dict_sum, decimals=3) 
                            for key,val in overlap_dict.items() if val/dict_sum > 10**-3}

            print(ind_prev)
            print(overlap_dict)
            district_df.at[ind_prev, 'inverse_population_overlap'] = overlap_dict
            
    return district_df

In [ ]:
# find if a district has changed between last year and this year
def check_if_districts_changed(this_year, district_df, threshold_for_change=0.1):
    """
    *** DEPRECIATED *** 
    might have some bugs. 
    use district_overlap to perform this function instead.
    
    Checks if each district changed more than the set threshold since the last election year.
    Change is a fraction of the district which overlaps with the previous year of that district (ranges 0-1).
    If the total area of the district increases, the overlap area is divided by the largest of the two years.
        (this prevents counting a district as unchanged if its area has increased)
    
    input:
        this_year -- (int) Elections years of the districts you want to check. Even years only. 
        district_df -- (pd dataframe) dataframe with default indices and shapefiles stored in 
                       a column named 'shape'
    output:
        district_df -- pandas data frame with index in format ST_00_YYYY (e.g. AL_01_2018) 
                       with change from last year stored in a column called 'border_change'.
                       Statuses:
                           'same' - this district has not changed at all since the previous year
                           'new' - indicates a district with this ID was not in this state last year
                           'changed' - indicates the borders have changed from last year, but that 
                                       this district was present in its state last year
    """
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]
        # loop over districts in your current year
        for ind,district in districts.iterrows():
            # find previous year's district
            district_prev = districts_prev.loc[districts_prev['district']==district['district']]
            ind_prev = '{}{}'.format(ind[:6],prev_year)

            # determine whether district is new or borders have changed
            if ind_prev in districts_prev.index: # if the district didn't exist last year
                # then this district is new this year
                district_df.loc[ind,'border_change'] = 'new'
            else: 
                # check if the borders have changed
                shape_prev = district_prev['shape'].values[0]
                shape = district['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate overlap percent
                area = shape.area # area of this district
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                
                if year==2016:
                    # we switch data sources in 2016
                    # Tiger Line districts include area over water, while UCLA districts do not.
                    # to avoid counting all of these districts as changed (because of the added 
                    # water area), we always divide by area_prev
                    frac_overlap = overlap_area/area_prev
                else:   
                    # fractional overlap between new and old district
                    # divide by the larger of the old or new district so that increasing district area 
                    # counts as a change
                    frac_overlap = overlap_area/np.max([area,area_prev]) 
                
                if (1.-frac_overlap) < threshold_for_change: 
                    # then district has not changed
                    district_df.loc[ind,'border_change'] = 'same'
                else:
                    # district has changed
                    district_df.loc[ind,'border_change'] = 'changed'
                    print(ind)
                    print(frac_overlap)
                    
    return district_df

# Run code here

In [ ]:
# get centroid coords for everything 
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

centroid_df = read_shapefiles(years, verbose=False)
for year in years:
     centroid_df = get_centroid(centroid_df)
pickle.dump(centroid_df.drop('shape',axis=1), open('centroid.p', 'wb'))

In [ ]:
# find out the area overlap and border change between this year's and last year's districts
# takes ~2hrs to run for all 8 years.

# you can only check a district if you have last year's data, so you read one more district than the data you make
years_to_read = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]
years_to_check = [2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

# read in the data (make a fresh df)
#overlap_df = read_shapefiles(years_to_read, verbose=False)
for year in years_to_check:
    overlap_df = district_overlap(year, overlap_df)
    pickle.dump(overlap_df.drop('shape', axis=1), open('overlap_frac_{}.p'.format(year), 'wb'))
pickle.dump(overlap_df.drop('shape', axis=1), open('overlap_frac.p', 'wb'))

In [ ]:
# estimate % population which derives from each previous district

# you can only check a district if you have last year's data, so you read one more district than the data you make
years_to_read = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]
years_to_check = [2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]

# read in the data (make a fresh df)
pop_df = read_shapefiles(years_to_read, verbose=False)
for year in years_to_check:
    pop_df = population_overlap(year, pop_df)
    pickle.dump(pop_df.drop('shape', axis=1), open('pop_frac_{}.p'.format(year), 'wb'))
pickle.dump(pop_df.drop('shape', axis=1), open('pop_frac.p', 'wb'))

In [ ]:
# Read in the files written here and add them to a master overlap set
all_overlap_data_df = pickle.load(open('overlap_frac.p','rb'))
all_overlap_data_df['centroid'] = pickle.load(open('centroid.p','rb'))['centroid']
all_overlap_data_df['population_overlap'] = \
    pickle.load(open('pop_frac.p','rb'))['population_overlap']
all_overlap_data_df['inverse_population_overlap'] = \
    pickle.load(open('inv_pop_frac.p','rb'))['inverse_population_overlap']
pickle.dump(all_overlap_data_df, open('all_overlap_data.p', 'wb'))